<center>
    <h1>DeepInsight EfficientNet-B3 NoisyStudent with PyTorch Lightning</h1>
    <img src="https://storage.googleapis.com/kaggle-markpeng/MoA/pytorch_lightning.png" alt="Pytorch Ligntning" width="300"/>
</center>
<br/>

This notebook demonstrates the inference of a fine-tuned EfficientNet-B3 NoisyStudent model using transformed 2D feature map images of MoA dataset. The model was trained under PyTorch Lightning architecture. The model files have been kept in private as a spirit of fair competition.

For more details about <em>DeepInsight Tranformation</em>, checkout the following notebook:

<a href="https://www.kaggle.com/markpeng/deepinsight-transform" target="_blank">https://www.kaggle.com/markpeng/deepinsight-transform</a>

10-folds Cross Validation Loss: **0.014932**

***

Please upvote or cite this notebook if you like it, thanks!

***

**Reference:**

Sharma, Alok, Edwin Vans, D. Shigemizu, Keith A. Boroevich and T. Tsunoda (2019). "_DeepInsight: A methodology to transform a non-image data to an image for convolution neural network architecture_," Scientific Reports, nature.com.

In [1]:
kernel_mode = True

import sys
if kernel_mode:
    sys.path.insert(0, "../input/iterative-stratification")
    sys.path.insert(0, "../input/pytorch-lightning")
    sys.path.insert(0, "../input/gen-efficientnet-pytorch")

import os
import numpy as np
import pandas as pd
import time
import random
import math
import pickle
from pickle import dump, load
import glob

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.cm import get_cmap
from matplotlib import rcParams

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torch.autograd import Function
import torch.optim as optim

from torch.nn import Linear, BatchNorm1d, ReLU
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional import classification

import geffnet

import cv2

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
sns.set(style="darkgrid")

import gc
gc.enable()

rand_seed = 1120

print(f"PyTorch Version: {torch.__version__}")
print(f"PyTorch Lightning Version: {pl.__version__}")

PyTorch Version: 1.6.0
PyTorch Lightning Version: 1.0.2


In [2]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/gen-efficientnet-pretrained/tf_efficientnet_*.pth /root/.cache/torch/hub/checkpoints/
!ls -la /root/.cache/torch/hub/checkpoints/

total 761336
drwxr-xr-x 2 root root      4096 Nov  8 05:52 .
drwxr-xr-x 3 root root      4096 Nov  8 05:52 ..
-rw-r--r-- 1 root root  21386709 Nov  8 05:52 tf_efficientnet_b0_ns-c0e6a31c.pth
-rw-r--r-- 1 root root  31516408 Nov  8 05:52 tf_efficientnet_b1_ns-99dd0c41.pth
-rw-r--r-- 1 root root  36801803 Nov  8 05:52 tf_efficientnet_b2_ns-00306e48.pth
-rw-r--r-- 1 root root  49385734 Nov  8 05:52 tf_efficientnet_b3_ns-9d44bf68.pth
-rw-r--r-- 1 root root  77995057 Nov  8 05:52 tf_efficientnet_b4_ns-d6313a46.pth
-rw-r--r-- 1 root root 122404944 Nov  8 05:52 tf_efficientnet_b5_ns-6f26d0cf.pth
-rw-r--r-- 1 root root 173239537 Nov  8 05:52 tf_efficientnet_b6_ns-51548356.pth
-rw-r--r-- 1 root root 266853140 Nov  8 05:52 tf_efficientnet_b7_ns-1dbc32de.pth


In [3]:
model_type = "b3"
pretrained_model = f"tf_efficientnet_{model_type}_ns"
experiment_name = f"deepinsight_efficientnet_v3_{model_type}"

dataset_folder = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA/"

model_info = {
    "model_path":
    f"../input/deepinsight-efficientnet-v3-b3/{experiment_name}"
    if kernel_mode else
    f"/workspace/Kaggle/MoA/completed/deepinsight_efficientnet_v3_b3/{experiment_name}"
}

num_workers = 2 if kernel_mode else 6
gpus = [0]

if model_type == "b0":
    batch_size = 128
    infer_batch_size = 256
    image_size = 224  # B0
    drop_rate = 0.2  # B0
    resolution = 224
elif model_type == "b3":
    batch_size = 48
    infer_batch_size = 512
    image_size = 300  # B3
    drop_rate = 0.3  # B3
    resolution = 300
elif model_type == "b5":
    batch_size = 8
    infer_batch_size = 16
    image_size = 456  # B5
    drop_rate = 0.4  # B5
    resolution = 456
elif model_type == "b7":
    batch_size = 2
    infer_batch_size = 4
    # image_size = 800  # B7
    image_size = 772  # B7
    drop_rate = 0.5  # B7
    resolution = 772

# DeepInsight Transform
perplexity = 5

drop_connect_rate = 0.2
fc_size = 512

# Load MoA Data

In [4]:
train_features = pd.read_csv(f"{dataset_folder}/train_features.csv",
                             engine='c')
train_labels = pd.read_csv(f"{dataset_folder}/train_targets_scored.csv",
                           engine='c')

train_extra_labels = pd.read_csv(
    f"{dataset_folder}/train_targets_nonscored.csv", engine='c')

test_features = pd.read_csv(f"{dataset_folder}/test_features.csv", engine='c')

sample_submission = pd.read_csv(f"{dataset_folder}/sample_submission.csv",
                                engine='c')

In [5]:
# Sort by sig_id to ensure that all row orders match
train_features = train_features.sort_values(
    by=["sig_id"], axis=0, inplace=False).reset_index(drop=True)
train_labels = train_labels.sort_values(by=["sig_id"], axis=0,
                                        inplace=False).reset_index(drop=True)
train_extra_labels = train_extra_labels.sort_values(
    by=["sig_id"], axis=0, inplace=False).reset_index(drop=True)

sample_submission = sample_submission.sort_values(
    by=["sig_id"], axis=0, inplace=False).reset_index(drop=True)

In [6]:
train_features.shape, train_labels.shape, train_extra_labels.shape

((23814, 876), (23814, 207), (23814, 403))

In [7]:
test_features.shape

(3982, 876)

In [8]:
category_features = ["cp_type", "cp_dose"]
numeric_features = [
    c for c in train_features.columns
    if c != "sig_id" and c not in category_features
]
all_features = category_features + numeric_features
gene_experssion_features = [c for c in numeric_features if c.startswith("g-")]
cell_viability_features = [c for c in numeric_features if c.startswith("c-")]
len(numeric_features), len(gene_experssion_features), len(
    cell_viability_features)

(873, 772, 100)

In [9]:
train_classes = [c for c in train_labels.columns if c != "sig_id"]
train_extra_classes = [c for c in train_extra_labels.columns if c != "sig_id"]
len(train_classes), len(train_extra_classes)

(206, 402)

# Feature Encoding
As we only have three metadata features, a quick manual encoding process is done. All features are normalized into the value range of [0, 1].

In [10]:
for df in [train_features, test_features]:
    df['cp_type'] = df['cp_type'].map({'ctl_vehicle': 0, 'trt_cp': 1})
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 0.5, 72: 1})

In [11]:
train_features["cp_type"].value_counts()

1    21948
0     1866
Name: cp_type, dtype: int64

In [12]:
train_features["cp_dose"].value_counts()

0    12147
1    11667
Name: cp_dose, dtype: int64

In [13]:
train_features["cp_time"].value_counts()

0.5    8250
1.0    7792
0.0    7772
Name: cp_time, dtype: int64

# DeepInsight Transform - t-SNE 2D Embeddings
Based on https://github.com/alok-ai-lab/DeepInsight, but with some corrections to the norm-2 normalization.

Most of the credits should be given to the original authors!

## Implementation

Checkout <a href="https://static-content.springer.com/esm/art%3A10.1038%2Fs41598-019-47765-6/MediaObjects/41598_2019_47765_MOESM1_ESM.pdf" target="_blank">DeepInsight paper supplementary information</a> for more details.

In [14]:
# Modified from DeepInsight Transform
# https://github.com/alok-ai-lab/DeepInsight/blob/master/pyDeepInsight/image_transformer.py

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt
import inspect


class DeepInsightTransformer:
    """Transform features to an image matrix using dimensionality reduction

    This class takes in data normalized between 0 and 1 and converts it to a
    CNN compatible 'image' matrix

    """
    def __init__(self,
                 feature_extractor='tsne',
                 perplexity=30,
                 pixels=100,
                 random_state=None,
                 n_jobs=None):
        """Generate an ImageTransformer instance

        Args:
            feature_extractor: string of value ('tsne', 'pca', 'kpca') or a
                class instance with method `fit_transform` that returns a
                2-dimensional array of extracted features.
            pixels: int (square matrix) or tuple of ints (height, width) that
                defines the size of the image matrix.
            random_state: int or RandomState. Determines the random number
                generator, if present, of a string defined feature_extractor.
            n_jobs: The number of parallel jobs to run for a string defined
                feature_extractor.
        """
        self.random_state = random_state
        self.n_jobs = n_jobs

        if isinstance(feature_extractor, str):
            fe = feature_extractor.casefold()
            if fe == 'tsne_exact'.casefold():
                fe = TSNE(n_components=2,
                          metric='cosine',
                          perplexity=perplexity,
                          n_iter=1000,
                          method='exact',
                          random_state=self.random_state,
                          n_jobs=self.n_jobs)
            elif fe == 'tsne'.casefold():
                fe = TSNE(n_components=2,
                          metric='cosine',
                          perplexity=perplexity,
                          n_iter=1000,
                          method='barnes_hut',
                          random_state=self.random_state,
                          n_jobs=self.n_jobs)
            elif fe == 'pca'.casefold():
                fe = PCA(n_components=2, random_state=self.random_state)
            elif fe == 'kpca'.casefold():
                fe = KernelPCA(n_components=2,
                               kernel='rbf',
                               random_state=self.random_state,
                               n_jobs=self.n_jobs)
            else:
                raise ValueError(("Feature extraction method '{}' not accepted"
                                  ).format(feature_extractor))
            self._fe = fe
        elif hasattr(feature_extractor, 'fit_transform') and \
                inspect.ismethod(feature_extractor.fit_transform):
            self._fe = feature_extractor
        else:
            raise TypeError('Parameter feature_extractor is not a '
                            'string nor has method "fit_transform"')

        if isinstance(pixels, int):
            pixels = (pixels, pixels)

        # The resolution of transformed image
        self._pixels = pixels
        self._xrot = None

    def fit(self, X, y=None, plot=False):
        """Train the image transformer from the training set (X)

        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
            y: Ignored. Present for continuity with scikit-learn
            plot: boolean of whether to produce a scatter plot showing the
                feature reduction, hull points, and minimum bounding rectangle

        Returns:
            self: object
        """
        # Transpose to get (n_features, n_samples)
        X = X.T

        # Perform dimensionality reduction
        x_new = self._fe.fit_transform(X)

        # Get the convex hull for the points
        chvertices = ConvexHull(x_new).vertices
        hull_points = x_new[chvertices]

        # Determine the minimum bounding rectangle
        mbr, mbr_rot = self._minimum_bounding_rectangle(hull_points)

        # Rotate the matrix
        # Save the rotated matrix in case user wants to change the pixel size
        self._xrot = np.dot(mbr_rot, x_new.T).T

        # Determine feature coordinates based on pixel dimension
        self._calculate_coords()

        # plot rotation diagram if requested
        if plot is True:
            # Create subplots
            fig, ax = plt.subplots(1, 1, figsize=(10, 7), squeeze=False)
            ax[0, 0].scatter(x_new[:, 0],
                             x_new[:, 1],
                             cmap=plt.cm.get_cmap("jet", 10),
                             marker="x",
                             alpha=1.0)
            ax[0, 0].fill(x_new[chvertices, 0],
                          x_new[chvertices, 1],
                          edgecolor='r',
                          fill=False)
            ax[0, 0].fill(mbr[:, 0], mbr[:, 1], edgecolor='g', fill=False)
            plt.gca().set_aspect('equal', adjustable='box')
            plt.show()
        return self

    @property
    def pixels(self):
        """The image matrix dimensions

        Returns:
            tuple: the image matrix dimensions (height, width)

        """
        return self._pixels

    @pixels.setter
    def pixels(self, pixels):
        """Set the image matrix dimension

        Args:
            pixels: int or tuple with the dimensions (height, width)
            of the image matrix

        """
        if isinstance(pixels, int):
            pixels = (pixels, pixels)
        self._pixels = pixels
        # recalculate coordinates if already fit
        if hasattr(self, '_coords'):
            self._calculate_coords()

    def _calculate_coords(self):
        """Calculate the matrix coordinates of each feature based on the
        pixel dimensions.
        """
        ax0_coord = np.digitize(self._xrot[:, 0],
                                bins=np.linspace(min(self._xrot[:, 0]),
                                                 max(self._xrot[:, 0]),
                                                 self._pixels[0])) - 1
        ax1_coord = np.digitize(self._xrot[:, 1],
                                bins=np.linspace(min(self._xrot[:, 1]),
                                                 max(self._xrot[:, 1]),
                                                 self._pixels[1])) - 1
        self._coords = np.stack((ax0_coord, ax1_coord))

    def transform(self, X, empty_value=0):
        """Transform the input matrix into image matrices

        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
                where n_features matches the training set.
            empty_value: numeric value to fill elements where no features are
                mapped. Default = 0 (although it was 1 in the paper).

        Returns:
            A list of n_samples numpy matrices of dimensions set by
            the pixel parameter
        """

        # Group by location (x1, y1) of each feature
        # Tranpose to get (n_features, n_samples)
        img_coords = pd.DataFrame(np.vstack(
            (self._coords, X.clip(0, 1))).T).groupby(
                [0, 1],  # (x1, y1)
                as_index=False).mean()

        img_matrices = []
        blank_mat = np.zeros(self._pixels)
        if empty_value != 0:
            blank_mat[:] = empty_value
        for z in range(2, img_coords.shape[1]):
            img_matrix = blank_mat.copy()
            img_matrix[img_coords[0].astype(int),
                       img_coords[1].astype(int)] = img_coords[z]
            img_matrices.append(img_matrix)

        return img_matrices

    def fit_transform(self, X, empty_value=0):
        """Train the image transformer from the training set (X) and return
        the transformed data.

        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
            empty_value: numeric value to fill elements where no features are
                mapped. Default = 0 (although it was 1 in the paper).

        Returns:
            A list of n_samples numpy matrices of dimensions set by
            the pixel parameter
        """
        self.fit(X)
        return self.transform(X, empty_value=empty_value)

    def feature_density_matrix(self):
        """Generate image matrix with feature counts per pixel

        Returns:
            img_matrix (ndarray): matrix with feature counts per pixel
        """
        fdmat = np.zeros(self._pixels)
        # Group by location (x1, y1) of each feature
        # Tranpose to get (n_features, n_samples)
        coord_cnt = (
            pd.DataFrame(self._coords.T).assign(count=1).groupby(
                [0, 1],  # (x1, y1)
                as_index=False).count())
        fdmat[coord_cnt[0].astype(int),
              coord_cnt[1].astype(int)] = coord_cnt['count']
        return fdmat

    @staticmethod
    def _minimum_bounding_rectangle(hull_points):
        """Find the smallest bounding rectangle for a set of points.

        Modified from JesseBuesking at https://stackoverflow.com/a/33619018
        Returns a set of points representing the corners of the bounding box.

        Args:
            hull_points : an nx2 matrix of hull coordinates

        Returns:
            (tuple): tuple containing
                coords (ndarray): coordinates of the corners of the rectangle
                rotmat (ndarray): rotation matrix to align edges of rectangle
                    to x and y
        """

        pi2 = np.pi / 2.

        # Calculate edge angles
        edges = hull_points[1:] - hull_points[:-1]
        angles = np.arctan2(edges[:, 1], edges[:, 0])
        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)

        # Find rotation matrices
        rotations = np.vstack([
            np.cos(angles),
            np.cos(angles - pi2),
            np.cos(angles + pi2),
            np.cos(angles)
        ]).T
        rotations = rotations.reshape((-1, 2, 2))

        # Apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)

        # Find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)

        # Find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)

        # Return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        rotmat = rotations[best_idx]

        # Generate coordinates
        coords = np.zeros((4, 2))
        coords[0] = np.dot([x1, y2], rotmat)
        coords[1] = np.dot([x2, y2], rotmat)
        coords[2] = np.dot([x2, y1], rotmat)
        coords[3] = np.dot([x1, y1], rotmat)

        return coords, rotmat

In [15]:
class LogScaler:
    """Log normalize and scale data

    Log normalization and scaling procedure as described as norm-2 in the
    DeepInsight paper supplementary information.
    
    Note: The dimensions of input matrix is (N samples, d features)
    """
    def __init__(self):
        self._min0 = None
        self._max = None

    """
    Use this as a preprocessing step in inference mode.
    """
    def fit(self, X, y=None):
        # Min. of training set per feature
        self._min0 = X.min(axis=0)

        # Log normalized X by log(X + _min0 + 1)
        X_norm = np.log(
            X +
            np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
            1).clip(min=0, max=None)

        # Global max. of training set from X_norm
        self._max = X_norm.max()

    """
    For training set only.
    """
    def fit_transform(self, X, y=None):
        # Min. of training set per feature
        self._min0 = X.min(axis=0)

        # Log normalized X by log(X + _min0 + 1)
        X_norm = np.log(
            X +
            np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
            1).clip(min=0, max=None)

        # Global max. of training set from X_norm
        self._max = X_norm.max()

        # Normalized again by global max. of training set
        return (X_norm / self._max).clip(0, 1)

    """
    For validation and test set only.
    """
    def transform(self, X, y=None):
        # Adjust min. of each feature of X by _min0
        for i in range(X.shape[1]):
            X[:, i] = X[:, i].clip(min=self._min0[i], max=None)

        # Log normalized X by log(X + _min0 + 1)
        X_norm = np.log(
            X +
            np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
            1).clip(min=0, max=None)

        # Normalized again by global max. of training set
        return (X_norm / self._max).clip(0, 1)

# Dataset

In [16]:
class MoAImageDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels, transformer):
        self.features = features
        self.labels = labels
        self.transformer = transformer

    def __getitem__(self, index):
        normalized = self.features[index, :]
        normalized = np.expand_dims(normalized, axis=0)

        # Note: we are setting empty_value=1 to follow the setup in the paper
        image = self.transformer.transform(normalized, empty_value=1)[0]

        # Resize to target size
        gene_cht = cv2.resize(image, (image_size, image_size),
                              interpolation=cv2.INTER_CUBIC)

        # Convert to 3 channels
        image = np.repeat(gene_cht[np.newaxis, :, :], 3, axis=0)

        return {"x": image, "y": self.labels[index, :]}

    def __len__(self):
        return len(self.features)


class TestDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels, transformer):
        self.features = features
        self.labels = labels
        self.transformer = transformer

    def __getitem__(self, index):
        normalized = self.features[index, :]
        normalized = np.expand_dims(normalized, axis=0)

        # Note: we are setting empty_value=1 to follow the setup in the paper
        image = self.transformer.transform(normalized, empty_value=1)[0]

        # Resize to target size
        gene_cht = cv2.resize(image, (image_size, image_size),
                              interpolation=cv2.INTER_CUBIC)

        # Convert to 3 channels
        image = np.repeat(gene_cht[np.newaxis, :, :], 3, axis=0)

        return {"x": image, "y": -1}

    def __len__(self):
        return len(self.features)

# Model Definition

In [17]:
# Reference: https://github.com/rwightman/gen-efficientnet-pytorch/blob/master/geffnet/efficientnet_builder.py#L672
def initialize_weight_goog(m, n='', fix_group_fanout=True):
    # weight init as per Tensorflow Official impl
    # https://github.com/tensorflow/tpu/blob/master/models/official/mnasnet/mnasnet_model.py
    if isinstance(m, nn.Conv2d):
        fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        if fix_group_fanout:
            fan_out //= m.groups
        m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        fan_out = m.weight.size(0)  # fan-out
        fan_in = 0
        if 'routing_fn' in n:
            fan_in = m.weight.size(1)
        init_range = 1.0 / math.sqrt(fan_in + fan_out)
        m.weight.data.uniform_(-init_range, init_range)
        m.bias.data.zero_()


def initialize_weight_default(m, n=''):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight,
                                 mode='fan_in',
                                 nonlinearity='linear')

In [18]:
class MoAEfficientNet(pl.LightningModule):
    def __init__(
            self,
            pretrained_model_name,
            training_set=(None, None),  # tuple
            valid_set=(None, None),  # tuple
            test_set=None,
            transformer=None,
            num_classes=206,
            in_chans=3,
            drop_rate=0.,
            drop_connect_rate=0.,
            fc_size=512,
            learning_rate=1e-3,
            weight_init='goog'):
        super(MoAEfficientNet, self).__init__()

        self.train_data, self.train_labels = training_set
        self.valid_data, self.valid_labels = valid_set
        self.test_data = test_set
        self.transformer = transformer

        self.backbone = getattr(geffnet, pretrained_model)(
            pretrained=True,
            in_chans=in_chans,
            drop_rate=drop_rate,
            drop_connect_rate=drop_connect_rate,
            weight_init=weight_init)

        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.backbone.classifier.in_features, fc_size,
                      bias=True), nn.ELU(),
            nn.Linear(fc_size, num_classes, bias=True))

        if self.training:
            for m in self.backbone.classifier.modules():
                initialize_weight_goog(m)

        # Save passed hyperparameters
        self.save_hyperparameters("pretrained_model_name", "num_classes",
                                  "in_chans", "drop_rate", "drop_connect_rate",
                                  "weight_init", "fc_size", "learning_rate")

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        x = batch["x"]
        y = batch["y"]
        x = x.float()
        y = y.type_as(x)
        logits = self(x)

        loss = F.binary_cross_entropy_with_logits(logits, y, reduction="mean")

        self.log('train_loss',
                 loss,
                 on_step=True,
                 on_epoch=True,
                 prog_bar=True,
                 logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x = batch["x"]
        y = batch["y"]
        x = x.float()
        y = y.type_as(x)
        logits = self(x)

        val_loss = F.binary_cross_entropy_with_logits(logits,
                                                      y,
                                                      reduction="mean")

        self.log('val_loss',
                 val_loss,
                 on_step=True,
                 on_epoch=True,
                 prog_bar=True,
                 logger=True)

        return val_loss

    def test_step(self, batch, batch_idx):
        x = batch["x"]
        y = batch["y"]
        x = x.float()
        y = y.type_as(x)
        logits = self(x)
        return {"pred_logits": logits}

    def test_epoch_end(self, output_results):
        all_outputs = torch.cat([out["pred_logits"] for out in output_results],
                                dim=0)
        print("Logits:", all_outputs)
        pred_probs = F.sigmoid(all_outputs).detach().cpu().numpy()
        print("Predictions: ", pred_probs)
        return {"pred_probs": pred_probs}

    def setup(self, stage=None):
        self.train_dataset = MoAImageDataset(self.train_data,
                                             self.train_labels,
                                             self.transformer)

        self.val_dataset = MoAImageDataset(self.valid_data, self.valid_labels,
                                           self.transformer)

        self.test_dataset = TestDataset(self.test_data, None, self.transformer)

    def train_dataloader(self):
        train_dataloader = DataLoader(self.train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=num_workers,
                                      pin_memory=True,
                                      drop_last=False)
        print(f"Train iterations: {len(train_dataloader)}")
        return train_dataloader

    def val_dataloader(self):
        val_dataloader = DataLoader(self.val_dataset,
                                    batch_size=infer_batch_size,
                                    shuffle=False,
                                    num_workers=num_workers,
                                    pin_memory=True,
                                    drop_last=False)
        print(f"Validate iterations: {len(val_dataloader)}")
        return val_dataloader

    def test_dataloader(self):
        test_dataloader = DataLoader(self.test_dataset,
                                     batch_size=infer_batch_size,
                                     shuffle=False,
                                     num_workers=num_workers,
                                     pin_memory=True,
                                     drop_last=False)
        print(f"Test iterations: {len(test_dataloader)}")
        return test_dataloader

    def configure_optimizers(self):
        print(f"Initial Learning Rate: {self.hparams.learning_rate:.6f}")
        optimizer = optim.Adam(self.parameters(),
                               lr=self.hparams.learning_rate,
                               weight_decay=weight_decay)

        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                         T_max=T_max,
                                                         eta_min=0,
                                                         last_epoch=-1)

        return [optimizer], [scheduler]

# Inference

In [19]:
kfolds = 10
skf = MultilabelStratifiedKFold(n_splits=kfolds,
                                shuffle=True,
                                random_state=rand_seed)

label_counts = np.sum(train_labels.drop("sig_id", axis=1), axis=0)
y_labels = label_counts.index.tolist()

In [20]:
def get_model(model_path, test_set, transformer):
    model = MoAEfficientNet.load_from_checkpoint(
        model_path,
        pretrained_model_name=pretrained_model,
        training_set=(None, None),  # tuple
        valid_set=(None, None),  # tuple
        test_set=test_set,
        transformer=transformer,
        drop_rate=drop_rate,
        drop_connect_rate=drop_connect_rate,
        fc_size=fc_size,
        weight_init='goog')

    model.freeze()
    model.eval()
    return model


def save_pickle(obj, model_output_folder, fold_i, name):
    dump(obj, open(f"{model_output_folder}/fold{fold_i}_{name}.pkl", 'wb'),
         pickle.HIGHEST_PROTOCOL)


def load_pickle(model_output_folder, fold_i, name):
    return load(open(f"{model_output_folder}/fold{fold_i}_{name}.pkl", 'rb'))

In [21]:
def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.mean(-logloss)

In [22]:
# Ensure Reproducibility
seed_everything(rand_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

kfold_submit_preds = np.zeros((test_features.shape[0], len(train_classes)))
for i, (train_index, val_index) in enumerate(
        skf.split(train_features, train_labels[y_labels])):
    print(f"Inferencing on Fold {i} ......")
    print(train_index.shape, val_index.shape)

    model_path = glob.glob(
        f'{model_info["model_path"]}/fold{i}/epoch*.ckpt')[0]

    test = test_features[all_features].copy().values

    # Load LogScaler (Norm-2 Normalization)
    scaler = load_pickle(model_info["model_path"], i, "log-scaler")
    test = scaler.transform(test)

    # Load DeepInsight Feature Map
    transformer = load_pickle(model_info["model_path"], i,
                              "deepinsight-transform")

    print(f"Loading model from {model_path}")
    model = get_model(model_path, test_set=test, transformer=transformer)

    trainer = Trainer(
        logger=False,
        gpus=gpus,
        distributed_backend="dp",  # multiple-gpus, 1 machine
        precision=16,
        benchmark=False,
        deterministic=True)
    output = trainer.test(model, verbose=False)[0]
    submit_preds = output["pred_probs"]
    kfold_submit_preds += submit_preds / kfolds

    del model, trainer, scaler, transformer, test
    torch.cuda.empty_cache()
    gc.collect()

Inferencing on Fold 0 ......
(21432,) (2382,)
Loading model from ../input/deepinsight-efficientnet-v3-b3/deepinsight_efficientnet_v3_b3/fold0/epoch14-train_loss_epoch0.014167-val_loss_epoch0.014794-image_size300-resolution300-perplexity5-fc512.ckpt


GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
Using native 16bit precision.


Test iterations: 8


Logits: tensor([[ -6.8477,  -7.5312,  -6.9883,  ...,  -6.0586,  -7.8633,  -7.2578],
        [-14.0703, -10.4219,  -8.7578,  ...,  -9.3125,  -6.2617,  -8.5938],
        [-22.1562, -21.9062, -13.7344,  ..., -15.9922, -15.3750, -15.7969],
        ...,
        [ -5.1914,  -6.4609,  -7.1055,  ...,  -7.3008,  -9.5469,  -7.1797],
        [ -5.8516,  -7.4336,  -6.6719,  ...,  -5.7734,  -9.1406,  -6.6953],
        [ -6.7422,  -7.4375,  -6.5859,  ...,  -6.9492,  -7.3906,  -7.0977]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[1.0605e-03 5.3596e-04 9.2173e-04 ... 2.3327e-03 3.8457e-04 7.0429e-04]
 [7.7486e-07 2.9802e-05 1.5724e-04 ... 9.0301e-05 1.9045e-03 1.8525e-04]
 [0.0000e+00 0.0000e+00 1.0729e-06 ... 1.1921e-07 2.3842e-07 1.1921e-07]
 ...
 [5.5351e-03 1.5612e-03 8.2016e-04 ... 6.7472e-04 7.1406e-05 7.6151e-04]
 [2.8667e-03 5.9080e-04 1.2646e-03 ... 3.0994e-03 1.0723e-04 1.2350e-03]
 [1.1787e-03 5.8842e-04 1.3781e-03 ... 9.5844e-04 6.1655e-04 8.2636e-04]]

Inferencing on Fol

GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
Using native 16bit precision.


...

Test iterations: 8


Logits: tensor([[ -6.1289,  -6.7695,  -7.0352,  ...,  -6.5195,  -7.6875,  -7.1680],
        [-11.1875, -11.0938,  -9.6875,  ..., -11.1328,  -9.7500,  -7.0625],
        [-21.0156, -21.2812, -13.7891,  ..., -18.0625, -18.3281, -15.8047],
        ...,
        [ -5.1680,  -5.7969,  -8.4219,  ...,  -7.5742,  -8.7969,  -6.8438],
        [ -6.5430,  -7.1328,  -6.8281,  ...,  -5.4258,  -7.4062,  -5.3945],
        [ -6.2578,  -6.9766,  -7.5820,  ...,  -6.7617,  -7.8242,  -6.7109]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[2.174e-03 1.147e-03 8.798e-04 ... 1.472e-03 4.582e-04 7.701e-04]
 [1.383e-05 1.520e-05 6.205e-05 ... 1.460e-05 5.829e-05 8.559e-04]
 [0.000e+00 0.000e+00 1.013e-06 ... 0.000e+00 0.000e+00 1.192e-07]
 ...
 [5.665e-03 3.027e-03 2.199e-04 ... 5.131e-04 1.512e-04 1.065e-03]
 [1.438e-03 7.977e-04 1.081e-03 ... 4.383e-03 6.070e-04 4.520e-03]
 [1.912e-03 9.327e-04 5.093e-04 ... 1.156e-03 3.998e-04 1.216e-03]]

Inferencing on Fold 9 ......
(21432,) (2382,)
Loading 

GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
Using native 16bit precision.


Test iterations: 8


Logits: tensor([[ -7.1719,  -7.2227,  -7.0117,  ...,  -6.1758,  -8.1406,  -6.4141],
        [-10.7578, -10.4297,  -7.7109,  ...,  -9.5312,  -8.4141,  -7.1797],
        [-25.1875, -25.1250, -13.6016,  ..., -17.9688, -15.6172, -15.4062],
        ...,
        [ -4.8906,  -5.6758,  -7.5586,  ...,  -7.4688,  -9.1719,  -6.7812],
        [ -6.2930,  -6.9219,  -6.5234,  ...,  -6.4180,  -7.1680,  -6.1758],
        [ -7.8906,  -8.2734,  -7.7461,  ...,  -6.6836,  -7.7852,  -6.1328]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[7.672e-04 7.296e-04 9.003e-04 ... 2.075e-03 2.913e-04 1.636e-03]
 [2.128e-05 2.956e-05 4.478e-04 ... 7.254e-05 2.217e-04 7.615e-04]
 [0.000e+00 0.000e+00 1.252e-06 ... 0.000e+00 1.788e-07 1.788e-07]
 ...
 [7.462e-03 3.416e-03 5.212e-04 ... 5.703e-04 1.039e-04 1.134e-03]
 [1.846e-03 9.851e-04 1.467e-03 ... 1.629e-03 7.701e-04 2.075e-03]
 [3.741e-04 2.551e-04 4.323e-04 ... 1.249e-03 4.158e-04 2.165e-03]]



In [23]:
oof_predictions = np.load(
    glob.glob(f'{model_info["model_path"]}/../oof*.npy')[0])
oof_loss = mean_logloss(oof_predictions, train_labels[train_classes].values)
print(f"OOF Validation Loss: {oof_loss:.6f}")

OOF Validation Loss: 0.014932


In [24]:
print(kfold_submit_preds.shape)
kfold_submit_preds

(3982, 206)


array([[1.52713060e-03, 1.10685825e-03, 1.06084347e-03, ...,
        1.84249878e-03, 9.05513763e-04, 1.04534626e-03],
       [4.31537628e-05, 8.86321068e-05, 8.21650028e-04, ...,
        2.83360481e-04, 1.10226870e-03, 1.14649534e-03],
       [0.00000000e+00, 0.00000000e+00, 8.82148743e-06, ...,
        4.76837158e-07, 1.19209290e-07, 1.78813934e-07],
       ...,
       [4.51016426e-03, 2.06649303e-03, 4.46975231e-04, ...,
        6.00874424e-04, 2.03669071e-04, 9.88006592e-04],
       [1.94239616e-03, 8.03887844e-04, 9.78171825e-04, ...,
        2.79200077e-03, 3.00407410e-04, 2.77268887e-03],
       [9.68992710e-04, 6.01291656e-04, 6.39379025e-04, ...,
        1.56396627e-03, 3.88622284e-04, 1.45578384e-03]])

In [25]:
np.mean(kfold_submit_preds)

0.0036081196166370447

In [26]:
submission = pd.DataFrame(data=test_features["sig_id"].values,
                          columns=["sig_id"])
submission = submission.reindex(columns=["sig_id"] + train_classes)
submission[train_classes] = kfold_submit_preds.clip(0, 1)
# Set control type to 0 as control perturbations have no MoAs
submission.loc[test_features['cp_type'] == 0, submission.columns[1:]] = 0
submission.to_csv('submission.csv', index=False)

In [27]:
submission

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [28]:
torch.cuda.empty_cache()
gc.collect()

20

# EOF
Thanks for reading through this notebook.